In [1]:
# Install specific version of scipy

# Libraries
from helpers import read_json_clean, save_json_clean, within_polygon
from nn_isochrones import get_isochrone_by_driving_distance, get_isochrone_by_walking_distance, nearest_neighbor
import folium
import platform
import pandas as pd
import seaborn as sns
import geopandas as gdp
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import functions to calculate nearest-neighbors

import overpy

In [ ]:
api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """)

for way in result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Nodes:")
    for node in way.nodes:
        print("    Lat: %f, Lon: %f" % (node.lat, node.lon))

Name: Rhedenstraße
  Highway: residential
  Nodes:
    Lat: 50.746874, Lon: 7.156307
    Lat: 50.746598, Lon: 7.156237
    Lat: 50.746477, Lon: 7.156199
    Lat: 50.746360, Lon: 7.156124
    Lat: 50.746263, Lon: 7.156023
    Lat: 50.746194, Lon: 7.155926
    Lat: 50.746139, Lon: 7.155843
Name: Von-Ketteler-Straße
  Highway: residential
  Nodes:
    Lat: 50.746905, Lon: 7.155786
    Lat: 50.747222, Lon: 7.155621
Name: Mutter-Barat-Straße
  Highway: residential
  Nodes:
    Lat: 50.747989, Lon: 7.156302
    Lat: 50.747934, Lon: 7.156229
    Lat: 50.747695, Lon: 7.155421
    Lat: 50.747605, Lon: 7.155159
    Lat: 50.747241, Lon: 7.154310
    Lat: 50.746939, Lon: 7.153634
Name: Stegerwaldstraße
  Highway: residential
  Nodes:
    Lat: 50.747853, Lon: 7.153685
    Lat: 50.747666, Lon: 7.154022
    Lat: 50.747673, Lon: 7.154122
    Lat: 50.747939, Lon: 7.154603
    Lat: 50.748182, Lon: 7.155178
    Lat: 50.748318, Lon: 7.155590
    Lat: 50.748362, Lon: 7.155646
Name: Alte Schulstraße
  Highw

In [2]:
center = [47.3735089, 8.5390344]
radius = 8000  # in meters

In [3]:
# Overpass turbo query
api = overpy.Overpass()
query = f"""
        [out:json];
        node(around:{radius},{center[0]},{center[1]})["amenity"~"restaurant|cafe|bar"];
        out;"""

result = api.query(query)
print(f"Found {len(result.nodes)} nodes with amenities.")
# Create a list of dictionaries to store the data
data = []
for node in result.nodes:
    #print(node.tags)
    data.append({
        'name': node.tags.get('name', 'n/a'),
        'lat': float(node.lat),
        'lon': float(node.lon),
        'amenity': node.tags.get('amenity', 'n/a')
    })
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
save_json_clean('test_isochrone.json', data)
# Display the DataFrame
#df.head()

Found 1933 nodes with amenities.
geocoding complete. data saved to test_isochrone.json


In [ ]:
isochrone = get_isochrone_by_driving_distance(center[0], center[1])
print(isochrone)
for i in df.index:
    polygon = isochrone['features'][0]['geometry']['coordinates'][0]
    #convert to list of tuples
    is_within = within_polygon(polygon, [df['lat'][i], df['lon'][i]])
    print(is_within)
    
#within_polygon(isochrone, df)


{'type': 'FeatureCollection', 'bbox': [8.420168, 47.251431, 8.680641, 47.479287], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 900.0, 'center': [8.5383693, 47.3739291], 'area': 124342677.72}, 'geometry': {'coordinates': [[[8.420556, 47.358399], [8.420371, 47.357694], [8.420251, 47.357067], [8.420168, 47.356177], [8.420174, 47.355447], [8.420218, 47.354808], [8.42031, 47.354251], [8.420574, 47.353285], [8.420812, 47.352661], [8.421191, 47.351857], [8.424364, 47.345481], [8.43125, 47.341807], [8.431666, 47.341533], [8.432095, 47.341276], [8.432902, 47.340835], [8.433804, 47.340414], [8.4347, 47.340043], [8.435754, 47.339688], [8.436783, 47.339396], [8.437449, 47.339242], [8.438099, 47.339115], [8.438753, 47.339008], [8.439386, 47.338913], [8.440504, 47.338813], [8.441638, 47.33876], [8.442489, 47.33876], [8.443015, 47.338777], [8.448165, 47.337088], [8.449379, 47.336408], [8.450954, 47.327798], [8.448328, 47.324968], [8.445702, 47.322138], [8.443214, 47.3194